# Course: Deep Learning
# Author: Sandro Camargo <sandrocamargo@unipampa.edu.br>
# Single Neuron Linear Regression Example
# Dataset: https://archive.ics.uci.edu/dataset/547/algerian+forest+fires+dataset

A Python library is a collection of related functions. A library contains bundles of encapsuated code which can be used repeatedly in different programs.

In [ ]:
# Import Libraries
import keras # Neural Network Library
from keras import layers # Layers to a neural network
from keras import optimizers # optimizers
import pandas as pd # Data Manipulation library
import numpy as np # Fast Numeric Computing library
import tensorflow as tf # Optimizers
import matplotlib.pyplot as plt # Plot library
from tensorflow.keras.utils import plot_model # Print the network

In [ ]:
# Loading dataset
data = pd.read_csv('https://archive.ics.uci.edu/ml/machine-learning-databases/00547/Algerian_forest_fires_dataset_UPDATE.csv', header=1, skiprows=[124,125,126,170], usecols=list(range(0,13)))
# About the parameters
# Header=1: column names (day, month, year, ...) are in the line 1 of this CSV file.
# skiprows=[124,125,126,170]: this lines, which not contains valid data, are not imported. If this parameter is missing, all lines are imported.
# usecols=list(range(0,13)): The last column, which is named Classes, is not imported. If this parameter is missing, all columns are imported.
# delimiter=",": when the delimiter among columns is not a ;

# inspecting columns and data types from "data" dataframe
data.info()

The dataset must be randomly splitted in two parts: training set and testing set. The main approaches to split are holdout and n-fold cross validation.
*   Training set is used for building (training) the model.
*   Testing set is used for testing the generalization ability of the model built.

Moreover, inputs($x$) and outputs($y$) must be splitted in each set.




In [ ]:
np.random.seed(1) # Random numbers will be ever the same
rnd = np.random.rand(len(data)) < 0.8 # Training set will contain 80% of the data

# Create the training dataset (80%)
train_data = data[rnd].copy()
train_x = train_data.drop(columns=data.columns[12])  # Drop the output column
train_y = train_data[data.columns[12]]  # Extract the output column

# Create the testing dataset (20%)
test_data = data[~rnd].copy()
test_x = test_data.drop(columns=data.columns[12])  # Drop the output column
test_y = test_data[data.columns[12]]  # Extract the output column

# Verifying dataset dimensions
print('The training dataset (inputs) dimensions are: ', train_x.shape)
print('The training dataset (outputs) dimensions are: ', train_y.shape)
print('The testing dataset (inputs) dimensions are: ', test_x.shape)
print('The testing dataset (outputs) dimensions are: ', test_y.shape)

After creating the datasets, the next step is defining the architecture of our model.

It must be defined:


*   Architecture: in terms of neurons and layers
*   Optimizer: is the algorithm or method used to change the weights in order to minimize the loss function.

The last step is compiling the model. In this step the loss function, the optimizer and the evaluation metrics must be defined.

In [ ]:
# Function to define model architecture
def build_model():
  # Defining the architecture
  # Sequential = Feedforward Neural Network
  # 1 single neuron
  # input_shape is the amount of columns from training set
  model = keras.Sequential([
        layers.Input(shape=[len(train_x.columns)]),
        layers.Dense(1)
  ])

  # Defining the optimizer
  optimizer = tf.keras.optimizers.RMSprop(
      learning_rate = 0.001)

  # Mean Squared Error (MSE) is the default loss function in regression models
  model.compile(loss = 'mse',
      optimizer = optimizer,
      metrics = ['mse','mae'])

  return model

Just for curiosity, you should observe how many parameters ($\theta$) your model has.
At this point, your model is built.

In [ ]:
model = build_model()
model.summary()

After creating the model, it must be trained (fitted).
Training is done using training set and the amount of epochs must be defined.

In [ ]:
EPOCHS = 500

history = model.fit(
    train_x, train_y, epochs = EPOCHS, verbose = 1
)

This plot should be generated just to inspect the learning convergence.
It is expected a decreasing of the loss function value through the epochs.


In [ ]:
plt.plot(history.history['mse'])
plt.title('Training MSE')
plt.ylabel('MSE')
plt.xlabel('Epoch')
plt.legend(['Error'], loc='upper right')
plt.show()

After the training process, the knowledge learnt by a neural network is stored in its weights.

In [ ]:
weights, biases = model.get_weights() # return a numpy list of weights
print(weights)
plt.plot(weights)
plt.ylabel('Weights')
plt.xlabel('Inputs')

In [ ]:
plt.barh(train_x.columns, weights[:,0].astype(float), align='center')
plt.xlabel("Weights")
plt.ylabel("Inputs")
#plt.title(target)
plt.savefig("NN-Weights.png")

After the training process, the model should be tested in order to measure its quality, it means, how good are its predictions. The model must be evaluated using the testing set, which is composed by samples that are not in the training set. In regression problems, the correlation coefficient is the default metric to measure the model quality.
The correlation coefficient is computed using real outputs ($y$) and predicted outputs ($\hat{y}$). Correlation coefficient can vary between 0 (bad predictions) and 1 (perfect predictions).

In [ ]:
test_predictions = model.predict(test_x).flatten() # predict radon activities with the built linear regression model

plt.scatter(test_y, test_predictions, marker = 'o', c = 'blue')
plt.plot([-5,35], [-5,35], color = 'black', ls = '--')
plt.ylabel('Predictions')
plt.xlabel('Real Values')
plt.title('Linear Regression with One Neuron (Testing Set)')
plt.ylim(-5, 35)
plt.xlim(-5, 35)
plt.axis(True)
plt.show()

# Convert test_y to a NumPy array
output = test_y.to_numpy()

# Calculate and print the correlation coefficient
correlation_coefficient = np.corrcoef(test_predictions, output.flatten())[0, 1]
print("Correlation Coefficient in testing set: %.4f" % correlation_coefficient)

In [ ]:
train_predictions = model.predict(train_x).flatten() # predict radom activities with the built linear regression model

plt.scatter(train_y, train_predictions, marker = 'o', c = 'blue')
plt.plot([-5,35], [-5,35], color = 'black', ls = '--')
plt.ylabel('Predictions')
plt.xlabel('Real Values')
plt.title('Linear Regression with One Neuron (Training Set)')
plt.ylim(-5, 35)
plt.xlim(-5, 35)
plt.axis(True)
plt.show()

# Convert test_y to a NumPy array
output = train_y.to_numpy()

# Calculate and print the correlation coefficient
correlation_coefficient = np.corrcoef(train_predictions, output.flatten())[0, 1]
print("Correlation Coefficient in testing set: %.4f" % correlation_coefficient)

In [ ]:
# Save the model architecture as an image
plot_model(model, to_file='model_architecture.png', show_shapes=True, show_layer_names=True)

